In [3]:
import numpy as np
from tqdm import trange
from gymnasium import spaces
from gymnasium.utils.env_checker import check_env

from flygym.simulation import SingleFlySimulation, Fly
from flygym.examples.common import PreprogrammedSteps
from flygym.examples.cpg_controller import CPGNetwork
from flygym.preprogrammed import get_cpg_biases
from hybrid_turning_fly import HybridTurningFly
from pathlib import Path
import matplotlib.pyplot as plt
from flygym import Parameters, Camera, SingleFlySimulation, Fly
from flygym.examples.turning_controller import HybridTurningNMF
from flygym.simulation import Simulation
from hybrid_turning_fly import HybridTurningFly
from movodor_arena import MovOdorArena
from flygym.preprogrammed import all_leg_dofs

seed = 1

class AbdomenFly(Fly):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    def _set_joints_stiffness_and_damping(self):
        super()._set_joints_stiffness_and_damping()
        
        # Set the abdomen joints stiffness and damping
        for body_name in ["A1A2", "A3", "A4", "A5", "A6"]:
            body = self.model.find("body", body_name)
            # add pitch degree of freedom to bed the abdomen
            body.add(
                "joint",
                name=f"joint_{body_name}",
                type="hinge",
                pos="0 0 0",
                axis="0 1 0",
                stiffness=5.0,
                springref=0.0,
                damping=5.0,
                dclass="nmf",)
            self.actuated_joints.append(f"joint_{body_name}")

    def _add_adhesion_actuators(self, gain):
        for body_name in ["A1A2", "A3", "A4", "A5", "A6"]:
            joint = self.model.find("joint", f"joint_{body_name}")
            actuator = self.model.actuator.add(
                "position",
                name=f"actuator_position_joint_{body_name}",
                joint=joint,
                forcelimited="true",
                ctrlrange="-1000000 1000000",
                forcerange="-10 10",
                kp=self.actuator_kp,
                dclass="nmf",
            )

            # this is needed if you do not want to override add joint sensors
            vel_actuator = self.model.actuator.add(
                "velocity",
                name=f"actuator_velocity_joint_{body_name}",
                joint=joint,
                dclass="nmf",
            )
            torque_actuator = self.model.actuator.add(
                "motor",
                name=f"actuator_torque_joint_{body_name}",
                joint=joint,
                dclass="nmf",
            )
            # self.actuated_joints.append(joint)
            self.actuators.append(actuator)

        return super()._add_adhesion_actuators(gain)
            


# Odor source: array of shape (num_odor_sources, 3) - xyz coords of odor sources
odor_source = np.array([[24, 10, 1.5]])

# Peak intensities: array of shape (num_odor_sources, odor_dimensions)
# For each odor source, if the intensity is (x, 0) then the odor is in the 1st dimension
# (in this case attractive). If it's (0, x) then it's in the 2nd dimension (in this case
# aversive)
peak_odor_intensity = np.array([[1, 0]])

# Marker colors: array of shape (num_odor_sources, 4) - RGBA values for each marker,
# normalized to [0, 1]
marker_colors = [[255, 127, 14]]
marker_colors = np.array([[*np.array(color) / 255, 1] for color in marker_colors])

odor_dimensions = len(peak_odor_intensity[0])
arena = MovOdorArena(
    size=(300, 300),
    friction=(1, 0.005, 0.0001),
    num_sensors=4,
    move_speed=0,
    move_direction="right",
    odor_source=odor_source,
    peak_intensity=peak_odor_intensity,
    diffuse_func=lambda x: x**-2,
    marker_colors=marker_colors,
    marker_size=0.3,
)

sim_params = Parameters(
    timestep=1e-4,
    render_mode="saved",
    render_playspeed=0.5,
    render_window_size=(800, 608),
    enable_olfaction=True,
    enable_adhesion=True,
    draw_adhesion=False,
    render_camera="Animat/camera_right",
)
timestep = sim_params.timestep
fly_female = AbdomenFly(
    name="female",
    
    enable_adhesion=True,
    enable_olfaction=True,
    spawn_pos=(0, 0, 0.2),
)

cam = Camera(
    fly=fly_female,
    camera_id="Animat/camera_right",
    window_size=(800, 608),
    play_speed=0.5,
)


sim = SingleFlySimulation(
    fly=fly_female,
    cameras=[cam],
    arena=arena,
    timestep=timestep,
)

preprogrammed_steps = PreprogrammedSteps()
legs = preprogrammed_steps.legs


run_time=1

all_joint_angles = []
kick_action={}
abdomen_action={}
target_num_steps = int(run_time / 1e-4)

kick_action["joint_RHCoxa"]=np.concatenate([np.linspace(0,1, target_num_steps//2),np.linspace(1, 0, target_num_steps//2)]).tolist()

kick_action["joint_RHCoxa_roll"]=np.linspace(0.7,0.7,target_num_steps).tolist()
kick_action["joint_RHCoxa_yaw"]=np.linspace(0,0,target_num_steps).tolist()
kick_action["joint_RHFemur"]=np.concatenate([np.linspace(2,2, target_num_steps//3),np.linspace(2,2.5, target_num_steps//3),np.linspace(2.5,1.7 , target_num_steps//3 +target_num_steps%3)]).tolist()
kick_action["joint_RHFemur_roll"]=np.linspace(0,0, target_num_steps).tolist()
kick_action["joint_RHTibia"]=np.concatenate([np.linspace(-1.0,-1.0, target_num_steps//3),np.linspace(-1.0,-2.0, target_num_steps//3),np.linspace(-2.0, 0, target_num_steps//3+target_num_steps%3)]).tolist()
kick_action["joint_RHTarsus1"]=np.concatenate([np.linspace(0.3,0.3 ,target_num_steps//2),np.linspace(0.3,0 , target_num_steps//2)]).tolist()
kick_action["joint_A1A2"]=np.linspace(0,0,target_num_steps).tolist()
kick_action["joint_A3"]=np.linspace(0,0,target_num_steps).tolist()
kick_action["joint_A4"]=np.linspace(0,-0.4,target_num_steps).tolist()
kick_action["joint_A5"]=np.linspace(0,-0.6,target_num_steps).tolist()
kick_action["joint_A6"]=np.linspace(0,0,target_num_steps).tolist()
swing_periods = preprogrammed_steps.swing_period

hindleg_stance_ids = np.linspace(swing_periods["RH"][1], 2 * np.pi, target_num_steps)
R_hindleg_start= preprogrammed_steps.get_joint_angles("RH", swing_periods["RH"][1])
R_hindleg_stretch=np.linspace(
    np.zeros(len(R_hindleg_start)), -R_hindleg_start, target_num_steps
)


run_time=1

adhesion_action = np.array([0.0 if leg.endswith("RH") else 1.0 for leg in legs])

timestep = sim_params.timestep
actuated_joints=["joint_RHCoxa","joint_RHCoxa_roll","joint_RHCoxa_yaw","joint_RHFemur","joint_RHFemur_roll","joint_RHTibia","joint_RHTarsus1","joint_A1A2","joint_A3","joint_A4","joint_A5","joint_A6"]
data_block = np.zeros((len(actuated_joints), target_num_steps))
abdomen_joints=["joint_A1A2","joint_A3","joint_A4","joint_A5"]
output_t=np.arange(target_num_steps)*timestep
input_t = np.linspace(0, timestep * (target_num_steps - 1), num=target_num_steps)

for i, joint in enumerate(actuated_joints):
    data_block[i,:] = np.interp(output_t, input_t, np.array(kick_action[joint]))
leg_action=[]

for i in range(target_num_steps):
    joint_angles = []
    for leg in legs:
        if leg.endswith("H"):
            if leg.startswith("R"):
                joint_angles.extend(data_block[:,i])
            elif leg.startswith("L"):
                joint_angles.extend(preprogrammed_steps.get_joint_angles(
                 leg,swing_periods[leg][1]            ))
           
        elif leg.endswith("F"):
           joint_angles.extend(preprogrammed_steps.get_joint_angles(leg, swing_periods[leg][1]            ))
        else:
             joint_angles.extend(preprogrammed_steps.get_joint_angles(
                 leg,swing_periods[leg][1]            ))
    
    
        
    
    action={"joints":joint_angles,"adhesion":adhesion_action}
    
    sim.step(action)
    sim.render()

from IPython.display import Video

cam.save_video("./outputs/kicking_video.mp4")

Video("./outputs/kicking_video.mp4", width=800, height=608)
